Download the dataset

In [104]:
import os
import requests
from tqdm import tqdm

download_url = "https://zenodo.org/record/6473001/files/ArtDL.zip"
local_file_name = "ArtDL.zip"

if os.path.exists(local_file_name):
  print(f"The file '{local_file_name}' already exists. Skipping download.")

else:
  print(f"Downloading the dataset from {download_url}...")
  
  head_response = requests.head(download_url)
  file_size = int(head_response.headers.get("content-length", 0))
  
  response = requests.get(download_url, stream=True)
  response.raise_for_status()
  
  with tqdm(total=file_size, unit="B", unit_scale=True, desc=local_file_name) as pbar:
    with open(local_file_name, "wb") as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)
        pbar.update(len(chunk))  # Update progress bar
  
  print(f"Dataset downloaded and saved as '{local_file_name}'")

The file 'ArtDL.zip' already exists. Skipping download.


Extract the zip file

In [105]:
import zipfile
import os

zip_file = "ArtDL.zip"

extract_dir = "dataset" 

if not os.path.exists(zip_file):
    print(f"The file '{zip_file}' does not exist. Please download it first.")
else:
    print(f"Extracting '{zip_file}' to '{extract_dir}'...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction complete. Files are saved in '{extract_dir}'")

Extracting 'ArtDL.zip' to 'dataset'...
Extraction complete. Files are saved in 'dataset'


Check what is contained in the dataset. Print and store images per classes. Also, images may have multiple classes.

In [106]:
import pandas as pd
import tabulate

csv_file_path = 'dataset/ArtDL/ArtDL.csv'

classes = [
    ("11H(ANTONY OF PADUA)", "ANTHONY OF PADUA"),
    ("11H(JOHN THE BAPTIST)", "JOHN THE BAPTIST"),
    ("11H(PAUL)", "PAUL"),
    ("11H(FRANCIS)", "FRANCIS OF ASSISI"),
    ("11HH(MARY MAGDALENE)", "MARY MAGDALENE"),
    ("11H(JEROME)", "JEROME"),
    ("11H(DOMINIC)", "SAINT DOMINIC"),
    ("11F(MARY)", "VIRGIN MARY"),
    ("11H(PETER)", "PETER"),
    ("11H(SEBASTIAN)", "SAINT SEBASTIAN")
]

def organize_df(df):
  column_mapping = {cls[0]: cls[1] for cls in classes}
  df = df.rename(columns=column_mapping).set_index("set")[sorted(column_mapping.values())].loc[["train", "val", "test"]]
  return df

df = pd.read_csv(csv_file_path)

columns_to_keep = [cls[0] for cls in classes]
df = df[columns_to_keep + ['item', 'set']]

df_normalized = df.copy()
df_normalized = df.drop(columns=['item']).groupby('set').sum().reset_index()
df_normalized = organize_df(df_normalized)

print("Table of classes:")
print(tabulate.tabulate(df_normalized, headers='keys', tablefmt='pretty'))
df_normalized.to_csv('1_ArtDL_classes.csv')
df[columns_to_keep] = df[columns_to_keep].astype(float)

# Weight the classes based on the number of items in each set
for index, row in df.iterrows():
  count_ones = row[columns_to_keep].sum()
  if count_ones > 0:
    df.loc[index, columns_to_keep] = row[columns_to_keep] / count_ones
  
df = df.drop(columns=['item']).groupby('set').sum().reset_index()

df[columns_to_keep] = df[columns_to_keep].astype(int)

df = organize_df(df)
print("Table of classes with weights:")
print(tabulate.tabulate(df, headers='keys', tablefmt='pretty'))
df.to_csv('1_ArtDL_classes_weighted.csv')

Table of classes:
+-------+------------------+-------------------+--------+------------------+----------------+------+-------+---------------+-----------------+-------------+
|  set  | ANTHONY OF PADUA | FRANCIS OF ASSISI | JEROME | JOHN THE BAPTIST | MARY MAGDALENE | PAUL | PETER | SAINT DOMINIC | SAINT SEBASTIAN | VIRGIN MARY |
+-------+------------------+-------------------+--------+------------------+----------------+------+-------+---------------+-----------------+-------------+
| train |       170        |       1220        |  1285  |       1497       |      1949      | 754  | 1471  |      387      |       628       |    15566    |
|  val  |        22        |        144        |  151   |       154        |      235       |  91  |  176  |      47       |       74        |    1920     |
| test  |        22        |        142        |  154   |       159        |      238       |  94  |  178  |      47       |       75        |    1913     |
+-------+------------------+------------

Download the list of images used for testing purposes

In [109]:
import os
import requests
import pandas as pd

# Download the test.txt file
test_txt_url = "https://raw.githubusercontent.com/iFede94/ArtDL/refs/heads/main/sets/test.txt"
test_txt_file = "2_test.txt"

response = requests.get(test_txt_url)
with open(test_txt_file, 'wb') as file:
  file.write(response.content)

with open(test_txt_file, 'r') as file:
  test_items = file.read().splitlines()

csv_file_path = 'dataset/ArtDL/ArtDL.csv'
df = pd.read_csv(csv_file_path)

missing_items = []
for item in test_items:
  if df[df['item'] == item].empty:
    missing_items.append(item)

if missing_items:
  print("Missing items:")
  for missing in missing_items:
    print(missing)
else:
  print("All items in test.txt exist in ArtDL.csv")

  
# Create ground truth file
image_dir = "dataset/ArtDL/JPEGImages"
missing_files = []

for item in test_items:
  file_name = f"{item}.jpg"
  if not os.path.exists(os.path.join(image_dir, file_name)):
    missing_files.append(file_name)

if missing_files:
  print("Missing image files:")
  for missing in missing_files:
    print(missing)
else:
  print("All image files exist in JPEGImages folder")

  num_rows = len(test_items)
  print(f"Number of rows in test.txt: {num_rows}")

All items in test.txt exist in ArtDL.csv
All image files exist in JPEGImages folder
Number of rows in test.txt: 1864


Create ground truth

In [110]:
import os
import json
import pandas as pd

classes = [
    ("11H(ANTONY OF PADUA)", "ANTHONY OF PADUA"),
    ("11H(JOHN THE BAPTIST)", "JOHN THE BAPTIST"),
    ("11H(PAUL)", "PAUL"),
    ("11H(FRANCIS)", "FRANCIS OF ASSISI"),
    ("11HH(MARY MAGDALENE)", "MARY MAGDALENE"),
    ("11H(JEROME)", "JEROME"),
    ("11H(DOMINIC)", "SAINT DOMINIC"),
    ("11F(MARY)", "VIRGIN MARY"),
    ("11H(PETER)", "PETER"),
    ("11H(SEBASTIAN)", "SAINT SEBASTIAN")
]

image_dir = "dataset/ArtDL/JPEGImages"
json_file_path = "2_ground_truth.json"
csv_file_path = "dataset/ArtDL/ArtDL.csv"
test_file = "2_test.txt"

df = pd.read_csv(csv_file_path)

ground_truth_data = []

with open(test_file, 'r') as file:
  test_items = file.read().splitlines()

# Process each image in the test file
for item in test_items:
  row = df[df['item'] == item]
  if row.empty:
    print(f"Warning: No matching row found in CSV for item '{item}'. Skipping...")
    continue
  
  row = row.iloc[0]

  # Find the column that is 1 and is in the classes list
  for cls in classes:
    if row[cls[0]] == 1:
      ground_truth_data.append({
        "item": item,
        "class": cls[1]
      })
      break

with open(json_file_path, 'w') as json_file:
  json.dump(ground_truth_data, json_file, indent=4)

print(f"Ground truth data has been saved to {json_file_path}")

Ground truth data has been saved to 2_ground_truth.json


Load images with Pillow

In [122]:
import json
from PIL import Image

# Open test.txt and read the lines
with open('2_test.txt', 'r') as file:
  test_items = file.read().splitlines()

images = []

for item in test_items:
  image_path = os.path.join('dataset/ArtDL/JPEGImages', f"{item}.jpg")
  try:
    image = Image.open(image_path)
    images.append(image)
  except Exception as e:
    print(f"Error loading image {image_path}: {e}")

print(f"Loaded {len(images)} images")


Loaded 1864 images


Test CLIP with these models:

openai/clip-vit-base-patch32
openai/clip-vit-base-patch16
openai/clip-vit-large-patch14


Process the images and see their probability against classes.
Use small batches (16 images)

In [140]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
from tqdm import tqdm
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "true"

model_name = "clip-vit-large-patch14"

# limit for testing purposes
#limit = 16 * 20
#images = images[:limit]

print(f"Number of images: {len(images)}")

# Load the model and processor
processor = AutoProcessor.from_pretrained(f'openai/{model_name}')
model = AutoModelForZeroShotImageClassification.from_pretrained(f'openai/{model_name}')

classes = [
    ("11H(ANTHONY OF PADUA)", "ANTHONY OF PADUA"),
    ("11H(JOHN THE BAPTIST)", "JOHN THE BAPTIST"),
    ("11H(PAUL)", "PAUL"),
    ("11H(FRANCIS)", "FRANCIS OF ASSISI"),
    ("11HH(MARY MAGDALENE)", "MARY MAGDALENE"),
    ("11H(JEROME)", "JEROME"),
    ("11H(DOMINIC)", "SAINT DOMINIC"),
    ("11F(MARY)", "VIRGIN MARY"),
    ("11H(PETER)", "PETER"),
    ("11H(SEBASTIAN)", "SAINT SEBASTIAN")
]


# Break images into smaller batches
batch_size = 16
images_batches = [images[i:i + batch_size] for i in range(0, len(images), batch_size)]

all_probs = []
with tqdm(total=len(images), desc="Processing Images", unit="image") as pbar:
    for batch_index, batch in enumerate(images_batches):
        try:
            # Process the batch
            inputs = processor(text=[cls[1] for cls in classes], images=batch, return_tensors="pt", padding=True)
            outputs = model(**inputs)
            
            # Get probabilities for the batch
            logits_per_image = outputs.logits_per_image  
            batch_probs = logits_per_image.softmax(dim=1)
            all_probs.append(batch_probs.detach())
            
            pbar.update(len(batch))
        except Exception as e:
            print(f"Error processing batch {batch_index + 1}: {e}")
            pbar.update(len(batch))

# Get one tensor with all the probabilities
all_probs = torch.cat(all_probs, dim=0)
print(f"Probabilities shape: {all_probs.shape}")

Number of images: 1864


Processing Images: 100%|██████████| 1864/1864 [08:48<00:00,  3.53image/s]

Probabilities shape: torch.Size([1864, 10])


In [139]:
import pandas as pd
import tabulate
from sklearn.metrics import average_precision_score

probs = all_probs

with open(json_file_path, 'r') as json_file:
  ground_truth_data = json.load(json_file)
ground_truth_dict = {item['item']: item['class'] for item in ground_truth_data}

# Initialize confusion matrix
confusion_matrices = {cls[1]: {'TP': 0, 'FP': 0, 'FN': 0} for cls in classes}
results_df = pd.DataFrame()

for i, item in enumerate(images):
  class_probs = all_probs[i]
  predicted_class = classes[class_probs.argmax().item()][1]
  true_class = ground_truth_dict.get(test_items[i], None)
  
  results_df = pd.concat([results_df, pd.DataFrame([{
      'File Name': os.path.basename(item.filename).split('.')[0],
      'Predicted Class': predicted_class,
      'True Class': true_class
  }])], ignore_index=True)

  for cls in classes:
    class_name = cls[1]
    if predicted_class == class_name and true_class == class_name:
      confusion_matrices[class_name]['TP'] += 1
    elif predicted_class == class_name and true_class != class_name:
      confusion_matrices[class_name]['FP'] += 1
    elif predicted_class != class_name and true_class == class_name:
      confusion_matrices[class_name]['FN'] += 1

# Store results if needed
results_df.to_csv('3_checked.csv')

performances_evaluation = pd.DataFrame(columns=["Class Name", "N Test Images", "Precision", "Recall", "F1-Score", "Average Precision"])
class_image_counts = {cls[1]: 0 for cls in classes}

for cls in classes:
  class_name = cls[1]
  tp = confusion_matrices[class_name]['TP']
  fp = confusion_matrices[class_name]['FP']
  fn = confusion_matrices[class_name]['FN']
  
  precision = tp / (tp + fp) if (tp + fp) > 0 else 0
  recall = tp / (tp + fn) if (tp + fn) > 0 else 0
  f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
  
  class_image_counts[class_name] = sum(1 for item in images if ground_truth_dict.get(os.path.basename(item.filename).split('.')[0]) == class_name)
  
  # Calculate average precision
  y_true = [1 if ground_truth_dict.get(os.path.basename(item.filename).split('.')[0]) == class_name else 0 for item in images]
  class_i = next(i for i, cls in enumerate(classes) if cls[1] == class_name)
  y_scores = [probs[i][class_i].item() for i in range(len(images))]
  avg_precision = average_precision_score(y_true, y_scores)
  
  performances_evaluation = pd.concat([performances_evaluation, pd.DataFrame([{
      "Class Name": class_name,
      "N Test Images": class_image_counts[class_name],
      "Precision": f"{precision:.2%}",
      "Recall": f"{recall:.2%}",
      "F1-Score": f"{f1_score:.2%}",
      "Average Precision": f"{avg_precision:.2%}"
  }])], ignore_index=True)

# Calculate mean values
mean_precision = performances_evaluation["Precision"].str.rstrip('%').astype(float).mean() / 100
mean_recall = performances_evaluation["Recall"].str.rstrip('%').astype(float).mean() / 100
mean_f1_score = performances_evaluation["F1-Score"].str.rstrip('%').astype(float).mean() / 100
mean_avg_precision = performances_evaluation["Average Precision"].str.rstrip('%').astype(float).mean() / 100

performances_evaluation = pd.concat([performances_evaluation, pd.DataFrame([{
  "Class Name": "MEAN",
  "N Test Images": "",
  "Precision": f"{mean_precision:.2%}",
  "Recall": f"{mean_recall:.2%}",
  "F1-Score": f"{mean_f1_score:.2%}",
  "Average Precision": f"{mean_avg_precision:.2%}"
}])], ignore_index=True)

# Reorder the dataframe based on the specified class order
class_order = ["ANTHONY OF PADUA", "FRANCIS OF ASSISI", "JEROME", "JOHN THE BAPTIST", "MARY MAGDALENE", "PAUL", "PETER", "SAINT DOMINIC", "SAINT SEBASTIAN", "VIRGIN MARY"]
performances_evaluation['Class Name'] = pd.Categorical(performances_evaluation['Class Name'], categories=class_order + ["MEAN"], ordered=True)
performances_evaluation = performances_evaluation.sort_values('Class Name').reset_index(drop=True)

os.makedirs('evaluations', exist_ok=True)

print("Performance Evaluation:")
print(tabulate.tabulate(performances_evaluation, headers="keys", tablefmt="pretty"))
performances_evaluation.to_csv(os.path.join('evaluations', f'{model_name}.csv'), index=False)



Performance Evaluation:
+----+-------------------+---------------+-----------+--------+----------+-------------------+
|    |    Class Name     | N Test Images | Precision | Recall | F1-Score | Average Precision |
+----+-------------------+---------------+-----------+--------+----------+-------------------+
| 0  | ANTHONY OF PADUA  |      14       |   4.71%   | 57.14% |  8.70%   |      11.41%       |
| 1  | FRANCIS OF ASSISI |      98       |  28.97%   | 63.27% |  39.74%  |      35.69%       |
| 2  |      JEROME       |      118      |   0.00%   | 0.00%  |  0.00%   |      14.76%       |
| 3  | JOHN THE BAPTIST  |      99       |  23.79%   | 69.70% |  35.48%  |      28.37%       |
| 4  |  MARY MAGDALENE   |      90       |  14.11%   | 88.89% |  24.35%  |      49.12%       |
| 5  |       PAUL        |      52       |   0.00%   | 0.00%  |  0.00%   |      11.87%       |
| 6  |       PETER       |      119      |   0.00%   | 0.00%  |  0.00%   |      14.97%       |
| 7  |   SAINT DOMINIC   |